Exercise 1 Simple neural network
Implement a simple neural network that predicts the class of iris plant basing on its parameters. 
Use 2 hidden layers. The layers should be fully connected with RELu activation function, except for the output layer, which should use softmax function.
 No regularisation or optimalisation are needed. No batching is needed. 


In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()


In [2]:
print(iris.DESCR)


.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [3]:
X = iris.data
y = iris.target

In [4]:
X_train , X_test , y_train, y_test = train_test_split(X,y , test_size=0.25 , random_state=42,shuffle=True)
train_labels = np.eye(3)[y] # one hot encoding 3 type of flowers


In [18]:
class Neuron():
    def __init__(self,in_features,out_features):
        self.weights = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (in_features,out_features))    # as in PyTorch
        self.bias = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (1,out_features))
        
    def forward(self, X):
        return X @ self.weights + self.bias
    


    

In [6]:
def hidden_unit(X , in_features, out_features):
    weights = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (in_features,out_features))    # as in PyTorch
    bias = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (1,out_features))
    
    return X @ weights + bias

def Relu_forward(X):
    return np.maximum(X,0)

def Relu_backward(X):
    # basicly kill gradient if <=0
    X[X<=0] = 0
    return X

def Softmax_forward(X):
    X_exp = np.exp(X)
    row_sum = np.sum(X_exp,axis=1,keepdims=True)
    return X_exp/row_sum  
def Logloss(X,labels):
    # matrix after softmax 
    return -np.mean(labels * np.log(X))

def CrossEntropyBackward(Softmax_output,labels):
    # combining log loss and soft max gives nice derivative 
    # X is output of softmax and labels are true labels
    return Softmax_output - labels     




In [7]:
N_1 = Neuron(4,10)
N_2 = Neuron(10,10)
N_3 = Neuron(10,3)


In [34]:


# initializing neurons for layers
N_1 = Neuron(4,10)
N_2 = Neuron(10,10)
N_3 = Neuron(10,3)
for step in range(10):
    # forward pary
    # 1st layer
    X_1 = N_1.forward(X)
    X_1_Relu = Relu_forward(X_1)


    # 2nd layer
    X_2 = N_2.forward(X_1_Relu)
    X_2 = Relu_forward(X_2)


    # final layer
    X_3 = N_3.forward(X_2)

    X_softmax = Softmax_forward(X_3)

    loss = Logloss(X_softmax, train_labels)

    # now backward part

    Z_softmax = CrossEntropyBackward(X_softmax,train_labels)

    Z_3_weigths = X_2.T @ Z_softmax    # weights for final layer
    Z_3_bias = Z_softmax           # + just plug forward gradient from previous layer

    # we have relu implemented correctly for chain rule so we just now apply 

    Z_2_relu = Relu_backward(Z_3_weigths)

    Z_2_weights = X_1_Relu @ Z_2_relu   # gradient for X_1 bias

    Z_2_bias = Z_2_relu            # gradient for X_2 bias

    Z_1_relu = Relu_backward(Z_2_weights)

    Z_1_weights = X.T @ Z_1_relu    # gradient for X_1 weights
    Z_1_bias = Z_1_relu             # gradient for X_1 bias


    # gradients applied in this form will reset every loop 
    # for now let's hardcode learning rate to 0.01 so we need to update in reverse gradient direction 


    N_1.weights -= 0.01 * Z_1_weights
    N_1.bias -= 0.01 * Z_1_bias

    N_2.weights -= 0.01 * Z_2_weights
    N_2.bias -= 0.01 * Z_2_bias

    N_3.weights -= 0.01 * Z_3_weigths
    N_3.bias -= 0.01 * Z_3_bias


    print(f"Step: {step}, loss value: {loss}")


        
        

ValueError: operands could not be broadcast together with shapes (4,10) (4,3) (4,10) 

In [37]:
N_1.bias.shape

(1, 10)

In [35]:
Z_1_bias

array([[  0.        ,  36.79189344,   7.16223393],
       [  0.        ,  38.81482945,   7.35224028],
       [  0.        ,  33.60200898,   6.68880142],
       [  0.        ,  34.32756882,   6.45390204],
       [  0.        ,  34.76670795,   7.01675858],
       [  0.        ,  39.45727029,   7.67755093],
       [  0.        ,  31.54138367,   6.6290364 ],
       [  0.        ,  36.92785938,   6.98001617],
       [  0.        ,  32.85238816,   6.2635941 ],
       [  0.        ,  38.16507512,   6.92789509],
       [  0.        ,  39.52814994,   7.5061303 ],
       [  0.        ,  35.03863984,   6.65232307],
       [  0.        ,  37.12889513,   6.856267  ],
       [  0.        ,  28.87600682,   6.11371939],
       [  0.        ,  40.35965883,   8.1902893 ],
       [  0.        ,  38.06740881,   8.15558337],
       [  0.        ,  37.09889472,   7.88827743],
       [  0.        ,  36.95263849,   7.30859547],
       [  0.        ,  43.970776  ,   8.07803604],
       [  0.        ,  35.18489

In [31]:
X_1_Relu.shape

(150, 10)